In [69]:
import pandas as pd
import requests
import json 
import folium 
import os
from datetime import datetime, timedelta

In [70]:
def sismo(url):
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')

    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 0,
            "minlatitude": -18.521, "maxlatitude": 0.132, "minlongitude": -84.419, "maxlongitude": -68.599, "limit": 20000}    
    data = requests.get(url, params = params)
    data = json.loads(data.text)
    return data

url = r"https://earthquake.usgs.gov/fdsnws/event/1/query?"
dic_sismo = sismo(url)
count=0
for i in (dic_sismo["features"]):
    if i["properties"]['cdi']:
        count += 1
# Convertir los datos a un dataframe
df_sismo = pd.json_normalize(dic_sismo['features'])

In [71]:
df_sismo.head()

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us6000kcsh,4.3,"11 km WSW of Pucara, Peru",1684408298865,1684411380040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",31.0,2.544,1.32,126.0,mb,earthquake,"M 4.3 - 11 km WSW of Pucara, Peru",Point,"[-71.5665, -16.0072, 131.861]"
1,Feature,us6000kcfu,4.0,"9 km WNW of Ayo, Peru",1684289721181,1684377795040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",17.0,3.320,0.66,154.0,mb,earthquake,"M 4.0 - 9 km WNW of Ayo, Peru",Point,"[-72.3568, -15.6528, 8.284]"
2,Feature,us6000kbqc,4.5,"6 km NW of Paján, Ecuador",1684004377115,1684005752040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",69.0,1.301,0.90,150.0,mb,earthquake,"M 4.5 - 6 km NW of Paján, Ecuador",Point,"[-80.4691, -1.5056, 43.612]"
3,Feature,us6000kbad,4.3,"31 km NE of Palca, Peru",1683854607800,1683856356040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",14.0,0.231,0.45,115.0,mb,earthquake,"M 4.3 - 31 km NE of Palca, Peru",Point,"[-69.721, -17.6075, 163.176]"
4,Feature,us6000kaam,5.2,"8 km W of Azogues, Ecuador",1683507115846,1683606904934,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,23.0,...,",dyfi,moment-tensor,origin,phase-data,",113.0,1.250,0.58,39.0,mww,earthquake,"M 5.2 - 8 km W of Azogues, Ecuador",Point,"[-78.9266, -2.7512, 89.603]"


In [72]:
df_sismo.columns

Index(['type', 'id', 'properties.mag', 'properties.place', 'properties.time',
       'properties.updated', 'properties.tz', 'properties.url',
       'properties.detail', 'properties.felt', 'properties.cdi',
       'properties.mmi', 'properties.alert', 'properties.status',
       'properties.tsunami', 'properties.sig', 'properties.net',
       'properties.code', 'properties.ids', 'properties.sources',
       'properties.types', 'properties.nst', 'properties.dmin',
       'properties.rms', 'properties.gap', 'properties.magType',
       'properties.type', 'properties.title', 'geometry.type',
       'geometry.coordinates'],
      dtype='object')

In [73]:
df_sismo.drop(columns=['type','properties.alert','properties.tz','properties.code','properties.url','properties.detail','properties.net'
                       ,'properties.status','properties.types','properties.sources','properties.magType','properties.type','geometry.type',
                       'properties.title'], axis=1, inplace=True)

In [74]:
df_sismo['Pais'] = df_sismo['properties.place'].str.split(',').str[1].str.strip()

df_sismo['properties.place'] = df_sismo['properties.place'].str.split(',', expand=True)[0]
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,Pais
0,us6000kcsh,4.3,11 km WSW of Pucara,1684408298865,1684411380040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,"[-71.5665, -16.0072, 131.861]",Peru


In [75]:
df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms") 
df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")

In [76]:
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,Pais
0,us6000kcsh,4.3,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,"[-71.5665, -16.0072, 131.861]",Peru


In [77]:
#Divido la columna geometry coordinates
df_sismo[['Longitud', 'Latitud', 'Profundidad']] = df_sismo['geometry.coordinates'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))

In [78]:
df_sismo["Longitud"] = df_sismo["Longitud"].astype(float) 
df_sismo["Latitud"] = df_sismo["Latitud"].astype(float) 
df_sismo["Profundidad"] = df_sismo["Profundidad"].astype(float)

In [79]:
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,Pais,Longitud,Latitud,Profundidad
0,us6000kcsh,4.3,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,"[-71.5665, -16.0072, 131.861]",Peru,-71.5665,-16.0072,131.861


In [80]:
df_sismo.drop("geometry.coordinates", axis=1, inplace=True) 

In [81]:
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,Pais,Longitud,Latitud,Profundidad
0,us6000kcsh,4.3,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,Peru,-71.5665,-16.0072,131.861


In [82]:
df_sismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9960 entries, 0 to 9959
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  9960 non-null   object        
 1   properties.mag      9960 non-null   float64       
 2   properties.place    9942 non-null   object        
 3   properties.time     9960 non-null   datetime64[ns]
 4   properties.updated  9960 non-null   datetime64[ns]
 5   properties.felt     1490 non-null   float64       
 6   properties.cdi      1490 non-null   float64       
 7   properties.mmi      485 non-null    float64       
 8   properties.tsunami  9960 non-null   int64         
 9   properties.sig      9960 non-null   int64         
 10  properties.ids      9960 non-null   object        
 11  properties.nst      3875 non-null   float64       
 12  properties.dmin     2799 non-null   float64       
 13  properties.rms      8869 non-null   float64     

In [83]:
df_sismo = df_sismo.rename(columns={'properties.mag':      'Magnituud',   
                                    'properties.place':    'Place',
                                    'properties.time':     'Fecha',  
                                    'properties.updated':  'Ultimo registro', 
                                    'properties.felt':     'Felt',
                                    'properties.cdi':      'cdi',
                                    'properties.mmi':      'mmi', 
                                    'properties.tsunami':  'Posibilidad tsunami',
                                    'properties.sig':      'Importancia del evento',
                                    'properties.ids':      'Ids',
                                    'properties.nst':      'nst',
                                    'properties.dmin':     'Distancia Horizontal Epicentro',
                                    'properties.rms':      'RMS',
                                    'properties.gap':      'Brecha Azimutal'})
df_sismo.head(1)

,id,Magnituud,Place,Fecha,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,Ids,nst,Distancia Horizontal Epicentro,RMS,Brecha Azimutal,Pais,Longitud,Latitud,Profundidad
0,us6000kcsh,4.3,11 km WSW of Pucara,2023-05-18 11:11:38.865,2023-05-18 12:03:00.040,NaN,NaN,NaN,0,284,",us6000kcsh,",31.0,2.544,1.32,126.0,Peru,-71.5665,-16.0072,131.861


In [84]:
df_sismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9960 entries, 0 to 9959
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              9960 non-null   object        
 1   Magnituud                       9960 non-null   float64       
 2   Place                           9942 non-null   object        
 3   Fecha                           9960 non-null   datetime64[ns]
 4   Ultimo registro                 9960 non-null   datetime64[ns]
 5   Felt                            1490 non-null   float64       
 6   cdi                             1490 non-null   float64       
 7   mmi                             485 non-null    float64       
 8   Posibilidad tsunami             9960 non-null   int64         
 9   Importancia del evento          9960 non-null   int64         
 10  Ids                             9960 non-null   object        
 11  nst 

In [85]:
#folder_path = "../DASHBOARD/CSV_TRANSFORMADOS"
#file_name = "Datos_Peru_USGS.csv"
#file_path = os.path.join(folder_path, file_name)
#
#df_sismo.to_csv(file_path, index=False)